In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# for the minimising function
from scipy.optimize import minimize

# for reading MATLAB files
from scipy.io import loadmat

In [3]:
# read data from file 

path = "D:\Programming\TestData\ex3data1.mat"
data = loadmat(path)
#loads X, y from MATLAB file ex3data1.mat
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

In [5]:
#extract X, y
X = data['X']
y = data['y']
X.shape, y.shape

((5000, 400), (5000, 1))

In [6]:
# unique class labels in y
np.unique(y)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [7]:
# add intercept term to X i.e. 1's column to X matrix

X = np.c_[np.ones((X.shape[0], 1)), X]
# using this method to avoid problems in the matrix multipications within the minimizing function

X.shape

(5000, 401)

In [8]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [9]:
def costFunctionReg(theta, X, y, learningRate):
    """
    X is (m, n+1) including intercept column
    y is (m, 1)
    theta is (n+1,1)
    """
    m = X.shape[0]
    h = sigmoid(np.dot(X, theta)) # h is hypothesis, h is (m,1) same as y
    
    term1 = np.dot(-y.T, np.log(h))
    term2 = np.dot((1-y).T, np.log(1-h))
    
    # intercept term is not regularized i.e. theta[0]=0, 
    # here shape of theta need not be retained, end result cost is a scalar
    regTerm = learningRate / (2 * m) * np.sum(np.square(theta[1:]))
    
    cost = (1/m) * np.sum(term1 - term2) + regTerm
    
    return cost

In [10]:
def gradientReg(theta, X, y, learningRate):
    """
    X is (m, n+1) including intercept column
    y is (m, 1)
    theta is (n+1,1)
    """    
    m = X.shape[0]
    h = sigmoid(np.dot(X, theta)) # h is hypothesis, h is (m,1) same as y
    
    # intercept term is not regularized
    # this is only for regularization term and here shape of theta needs to be maintained for end result grad
    theta[0] = 0
    regTerm = (learningRate / m) * theta
    
    grad = (1/m) * np.dot(X.T, (h-y)) + regTerm # (n+1,m) * (m,1) =>(n+1,1) => same as theta
    
    return grad

In [11]:
# testing costFunctionReg and gradientReg with theta set to zeros

testTheta = np.array([-2,-1,1,2])

XTest = np.concatenate((np.ones((5,1)),
np.fromiter((x/10 for x in range(1,16)), float).reshape((3,5)).T), axis = 1)

yTest = np.array([1,0,1,0,1])
lambdaTest = 3

cost = costFunctionReg(testTheta, XTest, yTest, lambdaTest)
grad = gradientReg(testTheta, XTest, yTest, lambdaTest)

# Expected cost: 
# 2.534819

#Expected gradients:
# 0.146561\n -0.548558\n 0.724722\n 1.398003

cost, grad

(2.534819396109744,
 array([ 0.14656137, -0.54855841,  0.72472227,  1.39800296]))

In [12]:
def one_vs_all(X, y, numLabels, learningRate):
    
    m = X.shape[0] # no of rows m
    params = X.shape[1] # no of features (no of thetas) n+1 (X already has intercept term)
    
    # return var
    allTheta = np.zeros((numLabels, params)) # (K, n+1) -> K labels/output classes, n features
        
    # labels are 1 - indexed not 0 indexed
    for i in range(1, numLabels + 1):   

        initialTheta = np.zeros((params, 1)) # (n+1,1)
        
        yClass = np.array([1 if label == i else 0 for label in y])
        # y has 10 classes (1 to 10) distributed among 400 rows, 
        # for each count 'i' set only corresponding element of yClass to 1
        # and all other elements to 0 
        # thus transforming into a binary logistic regression problem (yes/no)
        
        yClass = np.reshape(yClass, (m,1))
        # converting yClass from array (m,) to matrix (m,1)
        
        fmin = minimize(costFunctionReg, initialTheta, args = (X, yClass.ravel(), learningRate), 
                          method = 'TNC', jac = gradientReg, options = {'maxiter': 400})
        
        allTheta[i-1,:] = fmin.x
        
    return allTheta

In [13]:
learningRate = 0.1
# smaller lambda (learning rate), higher accuracy of final prediction 
# for lambda = 1, accuracy is 94.46%, for lambda = 0.1, accuracy is 96.46%
# however minimizing function takes more time for smaller lambda

numLabels = 10

allTheta = one_vs_all(X, y, numLabels, learningRate)

In [14]:
# checking output
allTheta.shape, X.shape, y.shape

((10, 401), (5000, 401), (5000, 1))

In [15]:
# checking output
allTheta

array([[-3.07209150e+00,  0.00000000e+00,  0.00000000e+00, ...,
         6.83994259e-03, -2.93583111e-10,  0.00000000e+00],
       [-3.73221215e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.33346949e-02, -2.55932029e-03,  0.00000000e+00],
       [-5.71267163e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.27868869e-05, -3.61375962e-07,  0.00000000e+00],
       ...,
       [-9.12551429e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.16739281e-04,  6.96212587e-05,  0.00000000e+00],
       [-5.62189442e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.12771539e-02,  8.58907140e-04,  0.00000000e+00],
       [-8.06799574e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.54044724e-05,  9.56083120e-07,  0.00000000e+00]])

In [16]:
def predictOneVsAll(allTheta, X):
    """
    X is (m,n+1)
    allTheta is (k,n+1)
    """
    # prediction is hypothesis    
    probs = sigmoid(np.dot(X, allTheta.T)) # (m,n+1) * (n+1,k) => (m,k)
    
    # create array of the index with the maximum probability
    hArgmax = np.argmax(probs, axis=1)
    # probs is (m,k) matrix. In each row, the index of the max element gives the label for that row
    # so in row 3, if 7th element is max, then row 3 is label 7
    
    # arrays are 0 indexed but labels are 1 indexed
    hArgmax = hArgmax + 1 # adding 1 to each element of array of type (m,)
    
    return hArgmax

In [17]:
yPred = predictOneVsAll(allTheta, X)

In [18]:
print('Training set accuracy: {} %'.format(np.mean(yPred == y.ravel())*100))

Training set accuracy: 96.46000000000001 %


In [19]:
# using SciKit Learn built in logistic regression functions

from sklearn.linear_model import LogisticRegression

In [20]:
clf = LogisticRegression(C=10, penalty='l2', solver='liblinear')
# Scikit-learn fits intercept automatically, so we exclude first column with 'ones' from X when fitting.
clf.fit(X[:,1:],y.ravel())

LogisticRegression(C=10, solver='liblinear')

In [21]:
pred2 = clf.predict(X[:,1:])
print('Training set accuracy: {} %'.format(np.mean(pred2 == y.ravel())*100))

Training set accuracy: 96.5 %
